In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import Normalize
import glob
import os

In [3]:
nc_files = sorted( glob.glob( r"C:\Users\locfa_v05v5qi\Documents\escadas_termohalinas\raw\Dataset\**\*.nc", recursive=True ) )


In [4]:
def subset_region(ds):
    # Ajustar longitudes se estiverem em 0–360
    if ds.lon.max() > 180:
        ds = ds.assign_coords(
            lon=((ds.lon + 180) % 360) - 180
        )

    # Criar máscara apenas para a dimensão n (perfis)
    mask = (
        (ds.lat <= 10) & (ds.lat >= -60) &
        (ds.lon >= -50) & (ds.lon <= -10)
    )

    if "Nobs" in ds.dims:
        ds = ds.isel(Nobs=mask)

    return ds
    
def check_staircase_exists(ml_h, gl_h): #verifica se os valores são fisicamente válidos (n só se existem)

    valid_ml = np.sum(~np.isnan(ml_h) & (ml_h > 0))
    valid_gl = np.sum(~np.isnan(gl_h) & (gl_h > 0))

    return (valid_ml >= 1) and (valid_gl >= 1)
    
def detect_staircases(ds):

    profile_dim = ds.lat.dims[0]
    n_profiles = ds.sizes[profile_dim] #p n ter problema d confundir "n" e "Nobs"

    staircase_sf = np.zeros(n_profiles, dtype=bool)
    staircase_dc = np.zeros(n_profiles, dtype=bool)

    for i in range(n_profiles):

        # =========================
        # SALT FINGER
        # =========================
        ml_mask_sf = ds.mask_ml_sf_layer.isel({profile_dim: i}) > 0
        gl_mask_sf = ds.mask_gl_sf_layer.isel({profile_dim: i}) > 0

        ml_h_sf = ds.ml_h.isel({profile_dim: i}).values[ml_mask_sf.values] #pega os valores onde ml_mask_sf é True 
        gl_h_sf = ds.gl_h.isel({profile_dim: i}).values[gl_mask_sf.values]

        if len(ml_h_sf) > 0 and len(gl_h_sf) > 0:
            staircase_sf[i] = check_staircase_exists(
                ml_h_sf, gl_h_sf
            )

        # =========================
        # DIFFUSIVE CONVECTION
        # =========================
        ml_mask_dc = ds.mask_ml_dc_layer.isel({profile_dim: i}) > 0
        gl_mask_dc = ds.mask_gl_dc_layer.isel({profile_dim: i}) > 0

        ml_h_dc = ds.ml_h.isel({profile_dim: i}).values[ml_mask_dc.values]
        gl_h_dc = ds.gl_h.isel({profile_dim: i}).values[gl_mask_dc.values]

        if len(ml_h_dc) > 0 and len(gl_h_dc) > 0:
            staircase_dc[i] = check_staircase_exists(
                ml_h_dc, gl_h_dc
            )

    ds["staircase_sf"] = (profile_dim, staircase_sf)
    ds["staircase_dc"] = (profile_dim, staircase_dc)

    return ds

In [6]:
#teste
# =========================================================
# INICIALIZAÇÃO DOS CONTADORES
# =========================================================

total_profiles = 0
total_profiles_region = 0
total_profiles_stair = 0

total_profiles_sf = 0
total_profiles_dc = 0
total_profiles_mixed = 0

# -------- CAMADAS --------
total_layers_sf = 0
total_layers_dc = 0
total_layers_total = 0

# -------- ESTATÍSTICAS VERTICAIS --------
all_ml_depth_sf = []
all_ml_depth_dc = []

all_ml_thickness_sf = []
all_ml_thickness_dc = []

n_layers_sf = []
n_layers_dc = []

# -------- PONTOS ESPACIAIS --------
all_points = []
all_points_stair = []
all_points_sf = []
all_points_dc = []
all_points_mixed = []

results = []

# =========================================================
# LOOP PRINCIPAL
# =========================================================

for file in nc_files:

    print(f"\nProcessando: {os.path.basename(file)}")

    ds = xr.open_dataset(file)

    # ---------------- PERFIS GLOBAIS ----------------
    n_global = ds.sizes.get("Nobs", 0)
    total_profiles += n_global
    print("Perfis globais:", n_global)

    # ---------------- RECORTE ESPACIAL ----------------
    ds = subset_region(ds)

    n_region = ds.sizes.get("Nobs", 0)
    total_profiles_region += n_region
    print("Perfis na região:", n_region)

    if n_region == 0:
        continue

    # ---------------- DETECÇÃO DE ESCADAS ----------------
    ds = detect_staircases(ds)

    sf_mask = ds.staircase_sf.values
    dc_mask = ds.staircase_dc.values

    stair_mask = sf_mask | dc_mask
    mixed_mask = sf_mask & dc_mask

    n_sf = int(sf_mask.sum())
    n_dc = int(dc_mask.sum())
    n_mixed = int(mixed_mask.sum())
    n_stair = int(stair_mask.sum())

    total_profiles_sf += n_sf
    total_profiles_dc += n_dc
    total_profiles_mixed += n_mixed
    total_profiles_stair += n_stair

    # =====================================================
    # ESTATÍSTICAS VERTICAIS E CONTAGEM DE CAMADAS
    # =====================================================

    for i in range(n_region):

        ml_p = ds.ml_p.isel(Nobs=i).values
        ml_h = ds.ml_h.isel(Nobs=i).values

        valid = (
            ~np.isnan(ml_p) &
            ~np.isnan(ml_h) &
            (ml_h > 0)
        )

        n_valid_layers = int(np.sum(valid))

        if n_valid_layers == 0:
            continue

        # -------- SF --------
        if sf_mask[i]:

            all_ml_depth_sf.extend(ml_p[valid])
            all_ml_thickness_sf.extend(ml_h[valid])
            n_layers_sf.append(n_valid_layers)

            total_layers_sf += n_valid_layers

        # -------- DC --------
        if dc_mask[i]:

            all_ml_depth_dc.extend(ml_p[valid])
            all_ml_thickness_dc.extend(ml_h[valid])
            n_layers_dc.append(n_valid_layers)

            total_layers_dc += n_valid_layers

        # -------- TOTAL --------
        if stair_mask[i]:
            total_layers_total += n_valid_layers

    # =====================================================
    # PONTOS ESPACIAIS
    # =====================================================

    lat = np.round(ds.lat.values, 2)
    lon = np.round(ds.lon.values, 2)

    all_points.extend(zip(lat, lon))
    all_points_stair.extend(zip(lat[stair_mask], lon[stair_mask]))
    all_points_sf.extend(zip(lat[sf_mask], lon[sf_mask]))
    all_points_dc.extend(zip(lat[dc_mask], lon[dc_mask]))
    all_points_mixed.extend(zip(lat[mixed_mask], lon[mixed_mask]))

    subset = ds[[
        "lat",
        "lon",
        "juld",
        "staircase_sf",
        "staircase_dc"
    ]]

    results.append(subset)

# =========================================================
# RESULTADOS FINAIS
# =========================================================

unique_points = set(all_points)
unique_points_stair = set(all_points_stair)
unique_points_sf = set(all_points_sf)
unique_points_dc = set(all_points_dc)
unique_points_mixed = set(all_points_mixed)

print("\n================ RESULTADOS ==================")

# ---------------- PERFIS ----------------
print("\n----- PERFIS -----")
print("Perfis globais:", total_profiles)
print("Perfis na região:", total_profiles_region)
print("Perfis com escada:", total_profiles_stair)
print("Perfis SF:", total_profiles_sf)
print("Perfis DC:", total_profiles_dc)
print("Perfis mistos:", total_profiles_mixed)

if total_profiles_region > 0:
    print("Frequência total de escadas:",
          round(100 * total_profiles_stair / total_profiles_region, 2), "%")

# ---------------- PONTOS ESPACIAIS ----------------
print("\n----- PONTOS ÚNICOS -----")
print("Pontos únicos na região:", len(unique_points))
print("Pontos únicos com escada:", len(unique_points_stair))
print("Pontos únicos SF:", len(unique_points_sf))
print("Pontos únicos DC:", len(unique_points_dc))
print("Pontos únicos mistos:", len(unique_points_mixed))

if len(unique_points) > 0:
    freq_spatial = len(unique_points_stair) / len(unique_points)
    print("Frequência espacial total:",
          round(freq_spatial * 100, 2), "%")

# ---------------- CAMADAS ----------------
print("\n----- QUANTIDADE TOTAL DE CAMADAS -----")
print("Total de camadas SF:", total_layers_sf)
print("Total de camadas DC:", total_layers_dc)
print("Total de camadas (qualquer escada):", total_layers_total)

# ---------------- ESTATÍSTICAS VERTICAIS ----------------
print("\n----- ESTATÍSTICAS VERTICAIS -----")

if all_ml_depth_sf:
    print("Profundidade média ML SF:",
          round(np.nanmean(all_ml_depth_sf), 1), "dbar")

if all_ml_depth_dc:
    print("Profundidade média ML DC:",
          round(np.nanmean(all_ml_depth_dc), 1), "dbar")

if all_ml_thickness_sf:
    print("Espessura média ML SF:",
          round(np.mean(all_ml_thickness_sf), 2), "dbar")

if all_ml_thickness_dc:
    print("Espessura média ML DC:",
          round(np.mean(all_ml_thickness_dc), 2), "dbar")

if n_layers_sf:
    print("Número médio de camadas por perfil SF:",
          round(np.mean(n_layers_sf), 2))

if n_layers_dc:
    print("Número médio de camadas por perfil DC:",
          round(np.mean(n_layers_dc), 2))


Processando: argo_00000000_00000249.nc
Perfis globais: 4
Perfis na região: 0

Processando: argo_00000250_00000499.nc
Perfis globais: 40
Perfis na região: 0

Processando: argo_00000500_00000749.nc
Perfis globais: 827
Perfis na região: 0

Processando: argo_00000750_00000999.nc
Perfis globais: 367
Perfis na região: 22

Processando: argo_00001000_00001249.nc
Perfis globais: 3017
Perfis na região: 198

Processando: argo_00001250_00001499.nc
Perfis globais: 3591
Perfis na região: 524

Processando: argo_00001500_00001749.nc
Perfis globais: 25540
Perfis na região: 6356

Processando: argo_00001750_00001999.nc
Perfis globais: 8247
Perfis na região: 1311

Processando: argo_00002000_00002249.nc
Perfis globais: 24
Perfis na região: 0

Processando: argo_00002250_00002499.nc
Perfis globais: 811
Perfis na região: 0

Processando: argo_00002500_00002749.nc
Perfis globais: 142
Perfis na região: 0

Processando: argo_00002750_00002999.nc
Perfis globais: 2786
Perfis na região: 0

Processando: argo_00003000

In [11]:
#poontos
only_sf = len(unique_points_sf) - len(unique_points_mixed)
only_dc = len(unique_points_dc) - len(unique_points_mixed)

total_union = len(unique_points_sf | unique_points_dc)

print("Apenas SF:", only_sf, round(100 * only_sf / total_union, 2), "%")
print("Apenas DC:", only_dc, round(100 * only_dc / total_union, 2), "%")
print("Mistos:", len(unique_points_mixed), round(100 * len(unique_points_mixed) / total_union, 2), "%")

Apenas SF: 6918 39.67 %
Apenas DC: 3615 20.73 %
Mistos: 6910 39.62 %
